In [1]:
#1. Captures images from pi
#2. Finds midpoint of peak - corresponds to centre of white line
#3. Uses PID controller but with midpoint of centre lane as reference - 2300



ImportError: No module named picamera.array

In [1]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera

from scipy.signal import find_peaks, butter, filtfilt

import time
import matplotlib.pyplot as plt
import skimage as ski

import cv2
import numpy as np
from PIL import Image


ImportError: No module named picamera.array

In [ ]:
# Camera resolution
res = (640, 480)

In [16]:
# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.02)

In [ ]:
#def crop_plot_peak(path, ROI, row):
def crop_plot_peak(Ibw, row):

    #This function reads an image with cv2.imread using path, crops it to ROI, and chooses the row to detect peaks
 
    #converting to black and white using L from pillow and displaying as grey
    #Ibw = Image.open(path).convert('L')
    
    plt.figure()
    plt.imshow(np.array(Ibw), cmap="gray")
    
    #converting pillow object to a numpy array
    Ibw_array = np.array(Ibw)
    
    #Ibw_array.shape
    
    '''#Cropping to Region of Interest
    ROI = np.array(Ibw_array[ROI[0]:ROI[1], ROI[2]:ROI[3]])
    ROI.shape
    plt.figure()
    plt.imshow(ROI)
    ROI.min(), ROI.max()
    L = ROI[row]'''
    
    L = Ibw_array[row]
    L.shape
    
    plt.figure()
    plt.plot(L)
    plt.show()
    
    return L



In [ ]:
def get_midpoint_peak(L):
    
    #smoothening both arrays to filter the noise in the image we use a 3rd order Butterworth filter
    # Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
    b, a = butter(3, 0.02)
    L_filt = filtfilt(b, a, L)
    L_shifted_filt = np.roll(L_filt, 1)
    L_shifted_filt[0] = L_shifted_filt[1]

    #Displaying L_filt
    plt.figure()
    plt.plot(L_filt)
    plt.show()
    
    #Displaying L_diff
    L_diff = L_filt - L_shifted_filt
    L_diff[0] = 0
    plt.figure()
    plt.plot(L_diff)
    plt.show()
    
    #Calculating the midpoint of the peak
    peak_start = np.argmax(L_diff)
    peak_stop = np.argmin(L_diff)
    peak_mid = (peak_start+peak_stop)/2
    print("Start of the peak is :", peak_start)
    print("End of the peak is :", peak_stop)
    print("Midpoint of the peak is :", peak_mid)
    
    return peak_mid

In [119]:
# Run a track detection algorithm on a single horizontal line.
# Uses YUV420 image format as the Y component corresponds to image intensity (gray image)
# and thus there is no need to convert from RGB to BW

camera = PiCamera()
        
# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 120

# Initialize the buffer and start capturing
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

# Measure the time needed to process 300 images to estimate the FPS
N = 300
k = 0
t = time.time()

for f in stream:
    # Get the intensity component of the image (a trick to get black and white images)
    I = f.array[:, :, 0]
    
    # Reset the buffer for the next image
    rawCapture.truncate(0)
    
    row = 400
    L = crop_plot_peak(I, row)
    midpoint_current = get_midpoint_peak(L)
    pos_diff = 2300 - midpoint_current  #2300 was the centre - hard coded
    #pwm1.duty_cycle = 1500000 + P *pos_diff 

    print("Position difference in the midpoint of the peaks is: ", pos_diff)
    # Increment the number of processed frames
    k += 1
    if k > N:
        break
        
time_elapsed = time.time() - t
print("Elapsed {:0.2f} seconds, estimated FPS {:0.2f}".format(time_elapsed, N / time_elapsed))

# Release resources
stream.close()
rawCapture.close()
camera.close()

Elapsed 13.17 seconds, estimated FPS 22.77
